In [1]:
import tensorflow as tf
import numpy as np
from sklearn.datasets import load_digits
import sys
import matplotlib.pyplot as plt

In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np




#导入数据
def read_and_decode(filename):
    #根据文件名生成一个队列
    filename_queue = tf.train.string_input_producer([filename])

    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)   #返回文件名和文件
    features = tf.parse_single_example(serialized_example,
                                       features={
                                           'label': tf.FixedLenFeature([], tf.int64),
                                           'img_raw' : tf.FixedLenFeature([], tf.string),
                                       })

    img = tf.decode_raw(features['img_raw'], tf.uint8)
    img = tf.cast(img,tf.float64)

    img = tf.reshape(img, [48, 24, 3])
    img=tf.split(img,3,2)[0]
    label = tf.cast(features['label'], tf.int32)
    img_batch, label_batch = tf.train.shuffle_batch([img, label],
                                                batch_size=64, capacity=25000,
                                                min_after_dequeue=10000)
    
    return img_batch, label_batch

def read_and_decode1(filename):
    #根据文件名生成一个队列
    filename_queue = tf.train.string_input_producer([filename])

    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)   #返回文件名和文件
    features = tf.parse_single_example(serialized_example,
                                       features={
                                           'label': tf.FixedLenFeature([], tf.int64),
                                           'img_raw' : tf.FixedLenFeature([], tf.string),
                                       })

    img = tf.decode_raw(features['img_raw'], tf.uint8)
    img = tf.cast(img,tf.float64)

    img = tf.reshape(img, [48, 24, 3])
    img=tf.split(img,3,2)[0]
    label = tf.cast(features['label'], tf.int32) 
    img_batch, label_batch = tf.train.batch([img, label], batch_size=64, 
                                            num_threads=1, capacity=64)
    return img_batch, label_batch

def read_and_decode1(filename):
    #根据文件名生成一个队列
    filename_queue = tf.train.string_input_producer([filename])

    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)   #返回文件名和文件
    features = tf.parse_single_example(serialized_example,
                                       features={
                                           'label': tf.FixedLenFeature([], tf.int64),
                                           'img_raw' : tf.FixedLenFeature([], tf.string),
                                       })

    img = tf.decode_raw(features['img_raw'], tf.uint8)
    img = tf.cast(img,tf.float64)

    img = tf.reshape(img, [48, 24, 3])
    img=tf.split(img,3,2)[0]
    label = tf.cast(features['label'], tf.int32)
    return img, label



def dr(a):
    s = list()
    for index, i in enumerate(a):
        s.append(list())
        for n in range(34):
            if n == i:
                s[index].append(1)
            else:
                s[index].append(0)            
    return s


"""
权重初始化,
初始化为一个接近0的很小的正数

"""


       
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev = 0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape = shape)
    return tf.Variable(initial)


#卷积:步长为1,0边距
def juanji(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding = 'VALID')


def chihua(x):
    return tf.nn.max_pool(x, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME')
    
#MNIST数据输入


#第一层 卷积层
x = tf.placeholder(tf.float32,[None, 48,24,1])

#最后一维代表通道，如果是rgb则为3 
x_image = tf.reshape(x, [-1, 48, 24, 1])
# 第一二参数值得卷积核尺寸大小，即patch，第三个参数是图像通道数，第四个参数是卷积核的数目，代表会出现多少个卷积特征图像;  
W_1 = weight_variable([5, 5, 1, 32])
# 对于每一个卷积核都有一个对应的偏置量。
b_1 = bias_variable([32])
# 图片乘以卷积核，并加上偏执量，
h_j1 = tf.nn.relu(juanji(x_image, W_1) + b_1)
h_c1 = chihua(h_j1)

In [3]:
#第二层 卷积层
# 32通道卷积，卷积出64个特征 
W_2 = weight_variable([5, 5, 32, 64])
# 64个偏执数据 
b_2 = bias_variable([64])
h_j2 = tf.nn.relu(juanji(h_c1, W_2) + b_2)
h_c2 = chihua(h_j2)

In [4]:
#第三层 全连接层
W_f1 = weight_variable([9 * 3 * 64, 1024])
b_f1 = bias_variable([1024])
h_c2_flat = tf.reshape(h_c2, [-1, 9 * 3 * 64])
h_f1 = tf.nn.relu(tf.matmul(h_c2_flat, W_f1) + b_f1)
keep_prob = tf.placeholder("float")
h_f1_drop = tf.nn.dropout(h_f1, keep_prob)




In [5]:
#第四层 Softmax输出层

W_f2 = weight_variable([1024, 34])
b_f2 = bias_variable([34])
logits = tf.matmul(h_f1_drop, W_f2) + b_f2
#Y = tf.nn.softmax(tf.matmul(h_f1_drop, W_f2) + b_f2)

In [6]:
y_ = tf.placeholder("float32", [None, 34])
#cross_entropy = -tf.reduce_sum(y_ * tf.log(Y + 1e-10)) # 定义交叉熵为loss函数    
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=logits)
train_step = tf.train.AdamOptimizer().minimize(cross_entropy) # 调用优化
saver = tf.train.Saver()
Y = tf.nn.softmax(logits)
correct_prediction = tf.equal(tf.argmax(Y,1), tf.argmax(y_,1))  
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
#img_batch ,label_batch = read_and_decode("train.tfrecords")
img_v_batch ,label_v_batch = read_and_decode("test.tfrecords")

In [7]:
#保存模型
#保存最优的三个
#幷记录每100次的正确率
#saver=tf.train.Saver(max_to_keep=3)
#max_train_accuracy=0
#f=open('model/acc.txt','w')

In [10]:
with tf.Session() as sess:
    writer = tf.summary.FileWriter('./logs', graph=sess.graph)
    sess.run(tf.global_variables_initializer())
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
#训练阶段
 
#    for i in range(5000):
#         img, label = sess.run([img_batch, label_batch]) 
#         #print(label[0])
#         label = dr(label)
#         #plt.imshow(img[0].reshape(48,24),cmap="gray")
#         #plt.show()
        
#         if i%100 == 0:  
#             train_accuracy = accuracy.eval(feed_dict={x:img, y_: label, keep_prob: 1.0})  
#             print("step %d, training accuracy %g"%(i, train_accuracy)) 
#             f.write(str(i+1)+', train_accuracy: '+str(train_accuracy)+'\n')
#             if train_accuracy>max_train_accuracy:
#                 max_train_accuracy = train_accuracy
#                 saver.save(sess,'model/mnist.ckpt',global_step=i+1)
#         train_step.run(feed_dict={x:img, y_: label, keep_prob: 0.5}) 
#     f.close()
  
    
    #模型的恢复
    model_file=tf.train.latest_checkpoint('model/')
    saver.restore(sess,model_file)
    #验证阶段
    m = 0
    Y_lab = list()
    y_lab = list()
    for i in range(34):
        Y_lab.append(0)
        y_lab.append(0)
        
    for i in range(59):
        img_v, label_v = sess.run([ img_v_batch, label_v_batch])
        yy = sess.run([Y], feed_dict = {x:img_v, keep_prob:1.0})
        label_vv = dr(label_v)
        validation_accuracy=accuracy.eval(session = sess,feed_dict = {x:img_v, y_:label_vv, keep_prob:1.0})
        
        for i in range(64):
            o=np.argmax(label_vv[i])
            y_lab[o]=y_lab[o]+1
            s = np.argmax(yy[0][i])
            if s == o:
                Y_lab[o] =Y_lab[o]+1
            
        print("test accuracy" ,validation_accuracy)
        m= m+ validation_accuracy
    print("正确率：",m/59)
    coord.request_stop()
    coord.join(threads)

test accuracy 0.75
test accuracy 0.859375
test accuracy 0.75
test accuracy 0.796875
test accuracy 0.859375
test accuracy 0.890625
test accuracy 0.890625
test accuracy 0.765625
test accuracy 0.75
test accuracy 0.71875
test accuracy 0.796875
test accuracy 0.78125
test accuracy 0.71875
test accuracy 0.75
test accuracy 0.75
test accuracy 0.875
test accuracy 0.8125
test accuracy 0.796875
test accuracy 0.796875
test accuracy 0.78125
test accuracy 0.734375
test accuracy 0.8125
test accuracy 0.796875
test accuracy 0.75
test accuracy 0.734375
test accuracy 0.765625
test accuracy 0.84375
test accuracy 0.75
test accuracy 0.8125
test accuracy 0.671875
test accuracy 0.78125
test accuracy 0.796875
test accuracy 0.703125
test accuracy 0.84375
test accuracy 0.75
test accuracy 0.796875
test accuracy 0.78125
test accuracy 0.8125
test accuracy 0.734375
test accuracy 0.703125
test accuracy 0.8125
test accuracy 0.828125
test accuracy 0.8125
test accuracy 0.796875
test accuracy 0.875
test accuracy 0.765625


In [12]:
def name(i):
    classes = {"0":0,"1":1,"2":2,"3":3,"4":4,"5":5,"6":6,"7":7,"8":8,"9":9,"A":10,"B":11,"C":12,"D":13,"E":14,"F":15,"G":16,"H":17,"J":18,"K":19,"L":20,"M":21,"N":22,"P":23,"Q":24,"R":25,"S":26,"T":27,"U":28,"V":29,"W":30,"S":31,"Y":32,"Z":33}
    for (key, value) in classes.items():
        if value == i:
            return key
        
        
for i in range(34):
    if y_lab[i] == 0:
        s = 0
    else:
        s  = Y_lab[i]/y_lab[i]
    print(name(i),"召回率：",s)

0 召回率： 0.6864406779661016
1 召回率： 0.8829787234042553
2 召回率： 0.55
3 召回率： 0.9732142857142857
4 召回率： 0.9310344827586207
5 召回率： 0.9347826086956522
6 召回率： 0.9586776859504132
7 召回率： 0.8365384615384616
8 召回率： 0.900709219858156
9 召回率： 0.9577464788732394
A 召回率： 0.8338983050847457
B 召回率： 0.7586206896551724
C 召回率： 0.9120879120879121
D 召回率： 0.4069767441860465
E 召回率： 0.9233333333333333
F 召回率： 0.8163265306122449
G 召回率： 0.35
H 召回率： 0.9121621621621622
J 召回率： 0.8105263157894737
K 召回率： 0.4696969696969697
L 召回率： 0.9358974358974359
M 召回率： 0.96875
N 召回率： 0.2459016393442623
P 召回率： 0.7945205479452054
Q 召回率： 0.9193548387096774
R 召回率： 0.8767123287671232
None 召回率： 0
T 召回率： 0.43564356435643564
U 召回率： 0.47619047619047616
V 召回率： 0.11320754716981132
W 召回率： 0.1875
S 召回率： 0.788235294117647
Y 召回率： 0.8644859813084113
Z 召回率： 0.9859154929577465
